# Introduction to EDA

## Course outline :

1. S6 (introduction) : what is EDA, pandas methods for data cleansing
2. S7 DataViz : matplotlib, seaborn, plotly, how to deal with outliers
3. S8 EDA recap, exercise
4. S9 correction
5. S10 regressions (linear, logistic)

## What is EDA

Presentation of EDA on [wikipedia.org](https://en.wikipedia.org/wiki/Exploratory_data_analysis) :
```
In statistics, exploratory data analysis (EDA) is an approach of analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell beyond the formal modeling and thereby contrasts with traditional hypothesis testing, in which a model is supposed to be selected before the data is seen. 
```

I would define data analysis as : collecting, transforming, and organizing data to formulate hypotheses, draw conclusions, make predictions, and support decision-making based on statistically proven facts.

![Data analysis cycle](./images/AnalysisCycle.png)

Typical data science workflow :

1. Get raw data (.csv, API/JSON, BDD/SQL,…)
2. Browse the data, read the columns/variables and make a working copy
3. Descriptive statistics: mean, median, max, min, standard deviation, quantiles, variance, covariances, correlation…
4. Cleansing: manage NaNs (delete? replace with mean, median?), convert other problematic values – such as inf and -inf–, delete columns (useless or unusable columns)
5. If necessary, assemble the data (merge), create the metrics we are missing (create new columns, transform…)
6. Visualize the data (distribution, linking) with graphs adapted to the data types (bar plots, scatter plots, cat plots, box plots…)
7. Identify and manage outliers (delete or transform)
8. Choose, train and test ML or analysis models (statistical inference)
9. Improve models (fine-tuning, feature selection) and comparison of best-fit models (A/B testing)
10. Deployment : forecasting with ML model, or dashboard for data storytelling / decision making)

The primary goal of EDA (Exploratory Data Analysis) is to build a good representation and, above all, *understanding* of the data by identifying how observations and variables are related to each other, highlighting trends and detecting anomalies. This is the first essential step, whatever is our objectives,  whether creating a complex machine learning model or simply testing the data to make a decision. It is from this exploration that you will formulate the hypotheses (on the relationships between variables) that will guide your work and that you will test.

From an operational point of view, you will :

- establishing basic statistics that can help us describe the data and observed trends
- generating relevant dataviz
- prepare the data for the upcoming analyses
- in particular, data cleansing : identifying errors, outliers and missing values, chose how to process them

A lot of tools can be relevant : 

![Tools](./images/Tools.png)

In this course we will focus on : 

* `pandas` and `numpy` for data cleansing and preparation 

* `matplotlib`, `seaborn`, `plotly` for dataviz

* `statsmodels` for simple analysis (regressions)

## `pandas` and `numpy` methods for data exploration and preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load and first look on data :

First, let’s import some data. The `seaborn` library has some training dataset. Let’s load a dataset and have a first look on it with the `.head()` method :

In [ ]:
tips_df = sns.load_dataset('tips')
tips_df.head()

A handful of methods of the `pandas` library are absolutely essential because they will allow us to carry out most of data exploration and cleaning. You have to know these methods to acquire an initial understanding of the data.

If you are not fluent with `pandas`, I recommand you to follow this [excellent notebook](https://github.com/virgilus/data-science/blob/main/Pandas_Introduction.ipynb) of my colleague [Virgile Pesce](https://github.com/virgilus). Especially if you need a refresher about methods like `.loc`/`.iloc`, `.groupby()`, `.merge()`, etc.

* `df.info()`: the first method to call once you have loaded your data in the form of a DataFrame. This method allows you to have the following information at once:

    * the size (*shape*) of the table (rows / columns)
    * the name of each column (name of *a priori* variables)
    * the number of non-null values in each column
    * the datatype of the values in each column
    * the size in memory of the DataFrame

In [ ]:
tips_df.info()

* for numeric/quantitative data: `df.describe()`. This method will provide you with the basic descriptive statistics (count, mean, standard deviation, median, min, max, quartiles, etc.) allowing you to represent the distribution.

In [ ]:
tips_df.describe()

In [ ]:
round(tips_df.describe(), 2)

* For categorical/qualitative variables: `df['column'].value_counts()`. This function allows you to quickly count the number of elements in each category (which are then enumerated at the same time). This advantageously replaces the use of the `df.unique()` and `df.nunique()` methods which would need more steps to give same information :

In [ ]:
tips_df['day'].value_counts()

In [ ]:
tips_wna_df = tips_df.copy()
tips_wna_df.iloc[1] = float('nan')
tips_wna_df.head()

In [ ]:
tips_wna_df['day'].value_counts(dropna=False, ascending=True)

In [ ]:
round(tips_df['day'].value_counts(normalize=True),2)

In [ ]:
tips_df.nunique()

In [ ]:
tips_df['day'].unique()

* a useful statistics is the mode `df.mode()` :

In [ ]:
tips_df['day'].mode()

If you want to get the mode for each columns, you have to use a *lambda function* with `.apply()`:

In [ ]:
tips_df.apply(lambda x: x.mode())

Lambda functions or anonymous functions are often used to implement operations that we will only perform once but on a certain number of elements. We will therefore declare a function on the fly that we will never call again afterwards. We could also have written :

In [ ]:
def get_mode(x):
    return x.mode()

tips_df.apply(get_mode)

That’s how we write custom treatment for data preparation !

### Cleansing

Now that we have had a first look on data, knowing columns names, datatypes and eventually if values are missing, we can do some cleansing.
It’s time to correct some issues :

* are the columns correctly named ?
* datatypes issues (generally its `datetime` issues : dates are in `string` format and not a `timestamp` type)
* sometime we may change the index, drop columns… (+ other time operation like periodicity)
* is there duplicated rows ?
* how to deal with missing values ?

#### Formating : rename columns

The method to rename columns is `.rename()`, which take dictionnary to associate old_name /new_name for columns. For example, if we find that `time` is not a very meaningful name, and want to rename this column `meal` :

In [ ]:
tips_df.rename(columns={'time': 'meal'}, inplace=True)
tips_df.columns

We may want to process all columns at once, for example by wanting to standardize naming formats :

In [ ]:
for c in tips_df.columns:
    tips_df.rename(columns={c: c.upper()}, inplace=True)
tips_df.columns

In [ ]:
for c in tips_df.columns:
    tips_df.rename(columns={c: c.lower()}, inplace=True)
tips_df.columns

#### Datatypes : date issue

Date formats and the datatype of variables that hold dates often cause problems with pandas

In [ ]:
dj_df = sns.load_dataset('dowjones')
dj_df.head()

In [ ]:
tips_df.info()

If you download real data that countains datetime values (for example [this dataset on Kaggle](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)) :

In [ ]:
orders_df = pd.read_csv('data/olist-dataset/olist_orders_dataset.csv')
orders_df.head()

In [ ]:
orders_df.info()

You notice that all date variable, (`order_purchase_timestamp`, `order_delivered_carrier_date`…) are of the `object` type (in fact, `string`) and not of the `timestamp` type. 

In [ ]:
orders_df['order_purchase_timestamp'] = pd.to_datetime(
    orders_df['order_purchase_timestamp'])

orders_df.info()

#### Index and dropping useless columns

One of the specificities of pandas is the existence of an index for rows which can be of any type. It can be interesting to identify one record by its timestamp.

* `.set_index()` method allows us to use a column as index :

In [ ]:
orders_df = orders_df.set_index(
    'order_purchase_timestamp').sort_index()

In [ ]:
orders_df.head()

* `.drop()` method gets rid of a column or a row :

In [ ]:
orders_df.drop(['order_id'], axis=1).head()

Don’t forget the flag `inplace` if you want to permanently modify the dataframe :

In [ ]:
orders_df.head()

Another formalism to drop columns is to use use the `columns` argument :

In [ ]:
orders_df.drop(columns=['order_id', 'customer_id'], inplace=True)
orders_df.head()

You can do the same with index (or `axis=0`):

In [ ]:
import datetime

orders_df.drop(index=datetime.datetime(2016, 9, 4, 21, 15, 19)).head()

We can perform complex exploration on the basis of datetime values, for example here we count the number of orders delivered weekly during year 2017 according to the carrier timestamp :

In [ ]:
delivered_weekly = (orders_df.sort_index()
     .query("order_status != 'delivered'")
     .loc['2017-01-01':'2017-12-31']
     .resample('W')['order_delivered_carrier_date']
     .count())
delivered_weekly

In [ ]:
delivered_weekly.plot();

#### Duplicated rows ?

Raw datafiles may contain a lot of errors, one of them being the duplication of records, for any reason.

* `.duplicated()` checks if there are duplicated rows :

In [ ]:
tips_df.duplicated()

In [ ]:
tips_df[tips_df.duplicated(keep=False)] # keep can be'first' or 'last'

In [ ]:
tips_df.duplicated().sum()

* `.drop_duplicates()` get rid of duclicate rows, `keep` being at `'first'` by default (don’t forget `inplace`) :

In [ ]:
tips_cleaned = tips_df.drop_duplicates()

In [ ]:
tips_cleaned.shape

In [ ]:
tips_df.shape

### Missing values

Missing values can have very negative effects on our analyses (poor estimation of the distribution, poor estimation of the parameters of a model, effects on the variance, etc.) like any other data defect, but also two specific negative effects:

- often missing values obey a pattern (faulty data collection, accident, difficulties in a place, a date, a target, in particular). If this pattern is not explained, it can contaminate the model or the analyses which will poorly reflect reality (statistical bias by not capturing an effect that nevertheless exists)

- if data is missing, it is as if our sampling was not complete, and therefore undermines our ability to highlight or identify significant relationships between variables in our analyses (loss of statistical power)

#### A typololgy of missing values

1. values that are missing completely randomly (example of cause : incident during the collection or copy of data)

2. values that are missing randomly (any value can be missing), but depending on the value of another variable, which is known (people with low level of education tend to ignore some questions, but not always)

3. values that are missing non-randomly (these are just given values that can be missing, for example we cannot record values greater than 10 and this variable had values greater than 10)

4. missing values for questions (often in questionnaires), measurements, or even specific variables within observations
entire records that are missing (not just a few variables, but a complete line), for example a questionnaire that is returned empty

We note that the context and the type of missing values will be very important in choosing the appropriate treatment. Moreover, for several cases, at this level of the course, we do not have the necessary knowledge to implement certain solutions. They are presented here just for documentation purposes. Come back to them when you are better equipped in terms of statistics and modeling.

#### Dealing with missing values

There are two strategies: deletion or replacement.

1. delete any record (row) where a value is missing, with the `df.dropna()` method. A radical and simple solution, but which risks to drastically reduce the sample size

2. for totally random missing values (case (1) in the list above), a good technique is to replace the missing value with the mean or mode (the most frequent value) of the variable concerned. This is called *imputation*. This can be done manually (define a function that replaces the missing values with, at choice, the mean or the mode), but the Scikit-Learn machine learning library has a whole set of methods for this – sklearn.inpute module. You will see this in you take a machine learning module, for the moment we will do these replacements “by hand”. This technique is problematic however in situations where the missing values are linked to another variable (cases (2) and (3) above).

3. If we suspect a pattern behind the missing values, we can consider creating a model that will allow, from the values taken by the other observed variables, to recreate the supposed values of the missing variable. We can use a regression model, a correlation model, or models that reconstruct the distribution of the missing value. Clustering models can also intervene (by identifying other records that "resemble" the one where the value is absent, but where it is present on the other hand and where we will copy it). There are other more complex models adapted to this objective: *multivariate imputation through chained equations*, *pattern mixture model*, *predictive mean matching*, etc. of course they are out of the scope of this introduction course.

* `.info()` method indicates count of non-null values in each columns. If yoou see a difference in the count between columns, this is the sign that there are missing values :

In [ ]:
orders_df.info()

* `.isna()` method gives the rows where a missing value is found :

In [ ]:
tips_df.isna()

In [ ]:
orders_df.isna().head()

* we can find out the count of missing value in each column by chaining with the `.sum()` method :

In [ ]:
orders_df.isna().sum()

* to do deeper investigation (e.g. : to see if there is a pattern) we can give a look at the rows containing missing values on a specific column with boolean indexing :

In [ ]:
orders_df[orders_df['order_approved_at'].isna()].head()

* `.dropna()` as its name suggests, drop rows with missing value. You can define :
    * a `subset` argument to select specific columns,
    * an `axis` too (drop columns or rows that countains missing values),
    * `how` defines the behavior of `.dropna` : if `how='any'` (default) a row or a column will be dropped if it contains at least one missing value, if `how='all'`, it will be dropped only if all values are missing (empty column or row)
    * `inplace` if we want to permanently modify the dataframe 

In [ ]:
orders_df.shape

In [ ]:
orders_df.dropna().shape

In [ ]:
orders_df.dropna(subset=['order_approved_at']).shape

In [ ]:
orders_df.dropna(axis=1).shape

In [ ]:
orders_df.dropna(axis=1, how='all').shape

In [ ]:
orders_df.dropna(axis=0, how='all').shape

In [ ]:
tips_wna_df.shape

In [ ]:
tips_wna_df.dropna(how='all').shape

Try to write a function that replace the missing values in a given column by the mode of this column.

Test it by replacing the missing values in `tips_wna_df` :

In [ ]:
def replace_NaN(df, column):

    # your code here

    return df


For this introductory course, we will limit ourselves to the first two solutions (drop rows with missing values, or replace the missing values by the mean or mode). 

Some advice: first, look for whether the absence of a value obeys a pattern, the most favorable situation being that these absences are totally random. Then ask yourself whether the affected variable is important for our analysis or not (no point in wasting time for nothing, with a poor quality measurement). This is where business knowledge, or domain knowledge is important. 

IMPORTANT : `.isna()`, `.dropna()` methods deal with the `NaN` datatypes. Sometimes missing values, errors, etc. are coded differently. For example if a column gives altitude, some points with errors of measurement may have a fixed negative "altitude" value (for example `-100000`). You have to inspect if such encoding is used in your data and find a way to deal with it. 

Keep a good record of all the manipulations you perform You will probably have to apply different methods: elimination in some, replacement by averages, ditto for outliers, etc. Cleaning is also an iterative process, it is extremely important to make your processing reproducible and traceable (know what you did, in what order, etc.). So never work directly on raw data (make copies !!!) and keep logs !!!

## Practice

To go further, we will need to do some dataviz which is the subject of the next course.

Right now you need to practice manipulating dataframes and the pandas methods that allow you to explore and clean them.

Here are some datasets that are freely available (downloadable), and that can be used as support to implement EDA techniques:

1. Synthesize the data with descriptive statistics
2. Identify if these data need to be cleaned (missing values? type problem?)
3. Make some simple `.plot()` or `.corr()` to see if some pattern emerges

Note : do not necessarally

### House prices

Sale price of real estate and many variables:
[house prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

This dataset is proposed as part of a machine learning topic, we will not go that far. We will just explore this data. Also, only three files interest us: train.csv and test.csv that will need to be assembled into a single dataframe (`.concatenate()`), and the text file that accompanies it to fully understand the data, which will allow you to formulate hypotheses to guide your exploration.

### Loan predications

Here is an interesting dataset to study the question of outliers that we will study later. For now, just explore and clean the dataset [loan predication](https://www.kaggle.com/datasets/ninzaami/loan-predication/data)

The problem is to relate characteristics of individuals, and the susceptibility to grant them a loan.

### Oil and gas US production 

his dataset shows US oil and gas production from June 2008 to June 2018: [us-oil-and-gas-production](https://www.kaggle.com/datasets/djzurawski/us-oil-and-gas-production-june-2008-to-june-2018)

This data includes temporal information.

In addition to classic explorations, this dataset is of interest:

* you will need to convert objects (string) into timestamps to be able to exploit this data
* with ```.groupby()``` try to calculate and represent annual gas and oil production (periodicity)
* try also to present production by state
* assemble into a single synthetic dataframe, by year, gas **and** oil production, for the whole of the US only, and another, state by state.

### Cars CO2 emissions

This dataset lists the amount of CO2 emitted for vehicles of different models: [vehicle-co2-emissions-dataset](https://www.kaggle.com/datasets/brsahan/vehicle-co2-emissions-dataset/data)

The opportunity to make some very informative visualizations! (next course)

### Real e-shop data

If you find the previous datasets too simple and do not require enough cleaning work, etc., here is a (real) dataset that will give you hard time: [Olist dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

A good exercise to learn how to merge data, and create metrics.

1. Observe the database schema carefully
2. From there, try to create a single dataframe (you will have to do ```.merge(), .groupby()```, etc.) that contains all the interesting data, according to hypotheses that you can make: for example, is there a correlation between the speed at which a consumer receives a product and the appreciation he leaves for a seller?